# Deep neural network for MNIST classification

Using tensorflow 2.0 to classify the MNIST data set of 70000 images of 28x28 pixels of handwritten digits per image.
The gols is to wite an algorithm to detect which digit is written. 

# Importing relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [2]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# as_supervised=True will load the dataset in a 2-tuple structure (input, target)
mnist_train, mnist_test= mnist_dataset['train'], mnist_dataset['test']

#number of validation samples
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

#number of test_samples
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)

#scale the data
def scale(image,label):
    # to make sure the value is a float
    image=tf.cast(image,tf.float32)
    # since the possible values for the inputs are 0 to 255
    image/=255.
    return image,label

# .map() allows to apply a custom transformation to a given dataset
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
# .take() method to take that many samples
validation_data = scaled_train_and_validation_data.take(num_validation_samples)
# .skip() as many samples as there are in the validation dataset
train_data = scaled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE=100
train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets= next(iter(validation_data))

# Model

In [3]:
input_size=784
output_size=10
hidden_layer_size=200

model=tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            
                           
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')                            
                            ])

## Optimizer and Loss

In [4]:

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [5]:
NUM_EPOCHS = 6
model.fit(train_data,epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets),verbose=2)


Epoch 1/6
540/540 - 7s - loss: 0.2639 - accuracy: 0.9239 - val_loss: 0.1426 - val_accuracy: 0.9572
Epoch 2/6
540/540 - 3s - loss: 0.1027 - accuracy: 0.9698 - val_loss: 0.1299 - val_accuracy: 0.9638
Epoch 3/6
540/540 - 4s - loss: 0.0667 - accuracy: 0.9799 - val_loss: 0.1193 - val_accuracy: 0.9650
Epoch 4/6
540/540 - 4s - loss: 0.0484 - accuracy: 0.9852 - val_loss: 0.1095 - val_accuracy: 0.9682
Epoch 5/6
540/540 - 4s - loss: 0.0380 - accuracy: 0.9879 - val_loss: 0.1110 - val_accuracy: 0.9702
Epoch 6/6
540/540 - 3s - loss: 0.0309 - accuracy: 0.9895 - val_loss: 0.1297 - val_accuracy: 0.9688


## Test the model 

In [6]:
test_loss, test_accuracy= model.evaluate(test_data)

1/1 [==============================] - 1s 630ms/step - loss: 0.1299 - accuracy: 0.9655


In [7]:
print('test loss: {0:.2f}. Test accuracy: {1:.2f}%' .format(test_loss, test_accuracy*100.))

test loss: 0.13. Test accuracy: 96.55%
